In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import numpy as np
import re
from numpy import dot
from numpy.linalg import norm
import math
import random

stop_words = set(stopwords.words('english')) 

In [3]:
snorkel_data_file_train = "/efs/CONSORT/CONSORT-TM/data/Methods_train.csv"
snorkel_data_file_test = "/efs/CONSORT/CONSORT-TM/data/Methods_test.csv"
snorkel_data_df_train = pd.read_csv(snorkel_data_file_train)
snorkel_data_df_test = pd.read_csv(snorkel_data_file_test)

snorkel_data_df = pd.concat([snorkel_data_df_train,snorkel_data_df_test])

print (len(snorkel_data_df))
display(snorkel_data_df.head(5))


# snorkel_data_file = "/efs/CONSORT2/data/snorkel_data/all_methodological_data_preprocessed_withSentenceSimilarityInfo_05262021.csv"

# snorkel_data_df = pd.read_csv(snorkel_data_file)
# display(snorkel_data_df.head(5))

# print (len(snorkel_data_df))

2460


,Unnamed: 0,PMCID,sentence_id,CONSORT_Item,labels,section,text
0,0,PMC3002766,S45,['3a'],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",Study design,"this double-blind, randomised, parallel-group,..."
1,1,PMC3002766,S46,['0'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Study design,the study was approved by local ethics committ...
2,2,PMC3002766,S47,['0'],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Study design,all patients gave written informed consent.
3,3,PMC3002766,S49,['4b'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",Settings and participants,the study was conducted from october 2003 to m...
4,4,PMC3002766,S50,['4a'],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Settings and participants,"recruited patients were ≥18 years old, with ss..."


In [4]:
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
import re

#define values for each label
#labels = [10','11a','11b','12a','12b','3a','3b','4a','4b','5','6a','6b','7a','7b','8a','8b','9', '0']

CONSORT_0 = -1
CONSORT_10= 0
CONSORT_11a =1
CONSORT_11b = 2
CONSORT_12a = 3
CONSORT_12b = 4
CONSORT_3a = 5
CONSORT_3b = 6
CONSORT_4a = 7
CONSORT_4b = 8
CONSORT_5 = 9
CONSORT_6a = 10
CONSORT_6b = 11
CONSORT_7a = 12
CONSORT_7b  = 13
CONSORT_8a = 14
CONSORT_8b = 15
CONSORT_9 = 16

#methods labeling functions
@labeling_function()
def regex_10(x):
    if ((re.search(r"random.*implementation", x["section_hierarchy"], flags=re.I))
    or (re.search(r"randomis", x["section_hierarchy"], flags=re.I))
    or (re.search(r"randomiz", x["section_hierarchy"], flags=re.I))):
        return CONSORT_10    
    else: 
        return CONSORT_0
    
@labeling_function()
def regex_11a(x):
    if ((re.search(r"blinding", x["section_hierarchy"], flags=re.I))
    or (re.search(r"masking", x["section_hierarchy"], flags=re.I))):
        return CONSORT_11a    
    else: 
        return CONSORT_0
    
@labeling_function()
def regex_11b(x):
    if ((re.search(r"similarity.*intervention", x["section_hierarchy"], flags=re.I)) 
    or (re.search(r"similarity.*treatment", x["section_hierarchy"], flags=re.I))):
        return CONSORT_11b  
    else: 
        return CONSORT_0
    
@labeling_function()
def regex_12a(x):
    if ((re.search(r"statistical", x["section_hierarchy"], flags=re.I)) 
    or (re.search(r"statistics", x["section_hierarchy"], flags=re.I))
    or (re.search(r"analys", x["section_hierarchy"], flags=re.I))
    or (re.search(r"data analys", x["section_hierarchy"], flags=re.I))):
        return CONSORT_12a  
    else: 
        return CONSORT_0   
    
@labeling_function()
def regex_12b(x):
    if (re.search(r"subgroup", x["section_hierarchy"], flags=re.I)):
        return CONSORT_12b  
    else: 
        return CONSORT_0  

@labeling_function()
def regex_3a(x):
    if (re.search(r"design", x["section_hierarchy"], flags=re.I)):
        return CONSORT_3a
    else: 
        return CONSORT_0  

@labeling_function()
def regex_3b(x):
    if ((re.search(r"change.*method", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*protocol", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*design", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*treatment", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*intervention", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*recruitment", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*eligibility", x["section_hierarchy"], flags=re.I)) 
        or (re.search(r"change.*plan", x["section_hierarchy"], flags=re.I))):
        return CONSORT_3b    
    else: 
        return CONSORT_0

@labeling_function()
def regex_4a(x):
    if ((re.search(r"participant", x["section_hierarchy"], flags=re.I))
        or (re.search(r"population", x["section_hierarchy"], flags=re.I))
        or (re.search(r"patient", x["section_hierarchy"], flags=re.I))
       or (re.search(r"inclusion", x["section_hierarchy"], flags=re.I))
       or (re.search(r"exclusion", x["section_hierarchy"], flags=re.I))
       or (re.search(r"subject", x["section_hierarchy"], flags=re.I))
       or (re.search(r"control group", x["section_hierarchy"], flags=re.I))
       or (re.search(r"eligibility", x["section_hierarchy"], flags=re.I))):
        return CONSORT_4a
    else: 
        return CONSORT_0  
    
@labeling_function()
def regex_4b(x):
    if ((re.search(r"recruitment", x["section_hierarchy"], flags=re.I))
        or (re.search(r"setting", x["section_hierarchy"], flags=re.I))):
        return CONSORT_4b
    else: 
        return CONSORT_0

@labeling_function()
def regex_5(x):
    if ((re.search(r"intervention", x["section_hierarchy"], flags=re.I))
        or (re.search(r"treatment", x["section_hierarchy"], flags=re.I))):
        return CONSORT_5
    else: 
        return CONSORT_0  
    
@labeling_function()
def regex_6a(x):
    if ((re.search(r"outcome", x["section_hierarchy"], flags=re.I))
        or (re.search(r"measure", x["section_hierarchy"], flags=re.I))
       or (re.search(r"assessment", x["section_hierarchy"], flags=re.I))
       or (re.search(r"measurement", x["section_hierarchy"], flags=re.I))
       or (re.search(r"endpoint", x["section_hierarchy"], flags=re.I))):
        return CONSORT_6a
    else: 
        return CONSORT_0  
    
@labeling_function()
def regex_6b(x):
    if ((re.search(r"change.*outcome", x["section_hierarchy"], flags=re.I)) 
    or (re.search(r"change.*endpoint", x["section_hierarchy"], flags=re.I))):
        return CONSORT_6b    
    else: 
        return CONSORT_0
    
@labeling_function()
def regex_7a(x):
    if (re.search(r"sample size", x["section_hierarchy"], flags=re.I)):
        return CONSORT_7a    
    else: 
        return CONSORT_0

@labeling_function()
def regex_7b(x):
    if ((re.search(r"interim", x["section_hierarchy"], flags=re.I))
        or (re.search(r"stopping", x["section_hierarchy"], flags=re.I))):
        return CONSORT_7b  
    else: 
        return CONSORT_0
        
@labeling_function()
def regex_8a(x):
    if ((re.search(r"sequence.*generation", x["section_hierarchy"], flags=re.I)) 
    or (re.search(r"allocation.*sequence", x["section_hierarchy"], flags=re.I))):
        return CONSORT_8a   
    else: 
        return CONSORT_0
    
@labeling_function()
def regex_8b(x):
    if ((re.search(r"randomis", x["section_hierarchy"], flags=re.I)) 
    or (re.search(r"randomiz", x["section_hierarchy"], flags=re.I))):
        return CONSORT_8b  
    else: 
        return CONSORT_0
    
@labeling_function()
def regex_9(x):
    if (re.search(r"concealment", x["section_hierarchy"], flags=re.I)):
        return CONSORT_9  
    else: 
        return CONSORT_0
    
def keyword_lookup(x, keywords, label):
    if any(word in x.text.lower() for word in keywords):
        return label
    return CONSORT_0

def make_keyword_lf(keywords, label):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label))


keyword_label_CONSORT_3a = make_keyword_lf(keywords=["randomized controlled trial","randomised controlled trial", 
"placebo controlled", "double blind", "parallel group",
"placebo-controlled","double-blind","parallel-group",
 "multicentre","multicenter"],label = CONSORT_3a)


keyword_label_CONSORT_3b = make_keyword_lf(keywords=["no longer feasible","decision was taken",
                                                     "decision was made","committee agreed"],label = CONSORT_3b)


keyword_label_CONSORT_4a = make_keyword_lf(keywords=["inclusion criteria","exclusion criteria","patients were eligible",
                                                     "aged","criteria were","history of","inclusion",
                                                     "excluded","eligible","for eligibility","following criteria", "criteria follow"],label = CONSORT_4a)

keyword_label_CONSORT_4b = make_keyword_lf(keywords=["study was conducted","conducted from","study area was",
                                                     "the study population","conducted in","carried out",
                                                     "enrolled in","enrolled into","came from","screened for",
                                                     "recruited patients from"," recruited from","medical center", "university center"],label = CONSORT_4b)

keyword_label_CONSORT_5 = make_keyword_lf(keywords=["matching placebo","regimen","matching",
                                                     "mg/kg","mg","infusion","study medication",
                                                     "medication","administered","open-label","patients were randomly",
                                                     "eligible patients","daily", 
                                                    "participant control group", "intervention group", "control groups"],label = CONSORT_5)


keyword_label_CONSORT_6a = make_keyword_lf(keywords=["primary outcome was","primary outcomes were", "primary outcomes",
                                                     "primary outcome","secondary outcomes","secondary outcome",
                                                     "secondary outcomes were","secondary outcome was","primary objective",
                                                     "adverse effects","illness effects","outcome assessment","outcome was",
                                                     "after randomisation","after randomization","objective","questionnaire",
                                                     "we defined","events","assessment","measurement","endpoint",
                                                     "primary endpoint","primary endpoints"],label = CONSORT_6a)


keyword_label_CONSORT_6b = make_keyword_lf(keywords=["original trial protocol","trial because of","early termination",
                                                     "the trial because","the original trial"],label = CONSORT_6b)

keyword_label_CONSORT_7a = make_keyword_lf(keywords=["sample size","power to detect","power","we estimated that","need a sample",
                                                     "sufficient power to","detect a difference","relative reduction",
                                                     "have sufficient power","dropout rate", "size calculation", "standard deviation",
                                                    "statistical power", "effect size", "correlation coefficient", "confidence interval"],label = CONSORT_7a)


keyword_label_CONSORT_7b = make_keyword_lf(keywords=["monitoring","monitoring committee","data monitoring","ethics committee",
                                                     "interim","stopping","cessation","trial cessation","stopping rule",
                                                     "interim analysis","interim analyses","committee judged",
                                                     "committee recommended","halt recruitment", "steering", "safety monitoring"],label = CONSORT_7b)


keyword_label_CONSORT_11a = make_keyword_lf(keywords=["to treatment allocation","masked to treatment","masked", 
                                                     "blinded to","blind to","blinding", "double-blinded to",
                                                     "masking","not have access", "blinded treatment allocation",
                                                      "blinded group allocation", "blinded group assignment", "blind treatment",
                                                      "blind group", "masked treatment", "masked group"
                                                     ],label = CONSORT_11a)

keyword_label_CONSORT_11b = make_keyword_lf(keywords=["tablets","numbered","identical","identically","match the size",
                                                     "manufactured to match"],label = CONSORT_11b)


keyword_label_CONSORT_12a = make_keyword_lf(keywords=["primary analysis","primary analyses","logistic regression","log-rank test",
                                                      "cox proportional","survival time","cox","curve","curves","log-rank",
                                                      "hazard","kaplan-meier","meier","kaplan","proportional","efficacy analysis",
                                                      "efficacy analyses","residual","intention-to-treat","linear","schoenfeld",
                                                      "comparisons","odds ratios","odds ratio","correlation","ratios","covariate",
                                                      "covariates","variables","changes from baseline","change from baseline",
                                                      "regression","exact test","fisher","binomial","two-sided","one-sided",
                                                     "statistically significant", "considered statistically", "statistical analyses",
                                                     "confidence interval", "anova", "spss",
                                                     "mean standard", "standard deviation", "variance", "pearson"],label = CONSORT_12a)

keyword_label_CONSORT_12b = make_keyword_lf(keywords=["subgroup analyses","subgroup analysis","subgroup","unplanned","exploratory",
                                                      "between subgroups","prespecified subgroup","prespecified subgroups"
                                                     ],label = CONSORT_12b)


keyword_label_CONSORT_8a = make_keyword_lf(keywords=["computer-generated","computer generated",
                                                            "block size","sequence generation", "allocation sequence",
                                                            "randomization sequence","randomisation sequence",
                                                            "generator","code was generated","allocation of randomization", 
                                                            "allocation of randomisation","random number sequence",
                                                              "random number sequences","random number generator",
                                                              "sequential allocation", "random number","randomization codes",
                                                              "randomisation codes","codes in blocks", "random allocation",
                                                    "allocation ratio", "web-based"],label = CONSORT_8a)

keyword_label_CONSORT_8b = make_keyword_lf(keywords=["block size","stratified by","stratified",
                                                    "by stratification","stratification","blocks of",
                                                    "computer-generated randomization",
                                                    "computer-generated randomisation",
                                                    "computer generated randomization","computer generated randomisation",
                                                    "computer-generated","computer generated","random number generator",
                                                    "random number sequence","random number sequences","codes in blocks",
                                                    "randomisation codes","randomization codes","random number", "intervention control"], label = CONSORT_8b)

keyword_label_CONSORT_9 = make_keyword_lf(keywords=["concealment","known only to","sealed envelopes","opaque", "numbered",
                                                             "sealed envelope","telephoning a","telephoning an",
                                                             "assigned box","assigned boxes",
                                                             "telephone randomisation service","telephone randomization service",
                                                             "packed in sealed","secure online","accessing a secure"], label = CONSORT_9)

keyword_label_CONSORT_10 = make_keyword_lf(keywords=["randomisation system","randomization system",
                                                               "web-based randomisation","web-based randomization",
                                                               "web randomisation","web randomization",
                                                               "online randomisation","online randomization",
                                                               "online web randomization","online web randomisation",
                                                               "minimisation","minimization","minimisation algorithm",
                                                               "web-based","randomization service",
                                                               "randomisation service","randomization program",
                                                               "randomization programme","randomisation program",
                                                               "randomisation programme"], label = CONSORT_10)

@labeling_function()
def similarity_lookup_CONSORT_10(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "10")):
        return CONSORT_10
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_11a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "11a")):
        return CONSORT_11a
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_11b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "11b")):
        return CONSORT_11b
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_12a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "12a")):
        return CONSORT_12a
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_12b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "12b")):
        return CONSORT_12b
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_3a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "3a")):
        return CONSORT_3a
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_3b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "3b")):
        return CONSORT_3b
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_4a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "4a")):
        return CONSORT_4a
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_4b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "4b")):
        return CONSORT_4b
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_5(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "5")):
        return CONSORT_5
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_6a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "6a")):
        return CONSORT_6a
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_6b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "6b")):
        return CONSORT_6b
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_7a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "7a")):
        return CONSORT_7a
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_7b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "7b")):
        return CONSORT_7b
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_8a(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "8a")):
        return CONSORT_8a
    else:
        return CONSORT_0
    
@labeling_function()
def similarity_lookup_CONSORT_8b(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "8b")):
        return CONSORT_8b
    else:
        return CONSORT_0

@labeling_function()
def similarity_lookup_CONSORT_9(x):
    if ((x["highest_frequency_label_in_top10"] == x["highest_similarity_label"]) and
    (x["highest_frequency_label_in_top10"] == "9")):
        return CONSORT_9
    else:
        return CONSORT_0
    

In [5]:
from snorkel.labeling import PandasLFApplier

lfs = [similarity_lookup_CONSORT_10,
      similarity_lookup_CONSORT_11a,
      similarity_lookup_CONSORT_11b,
      similarity_lookup_CONSORT_12a,
      similarity_lookup_CONSORT_12b,
      similarity_lookup_CONSORT_3a,
      similarity_lookup_CONSORT_3b,
      similarity_lookup_CONSORT_4a,
      similarity_lookup_CONSORT_4b,
      similarity_lookup_CONSORT_5,
      similarity_lookup_CONSORT_6a,
      similarity_lookup_CONSORT_6b,
      similarity_lookup_CONSORT_7a,
      similarity_lookup_CONSORT_7b,
      similarity_lookup_CONSORT_8a,
      similarity_lookup_CONSORT_8b,
      similarity_lookup_CONSORT_9,
       keyword_label_CONSORT_10,
    keyword_label_CONSORT_11a,
    keyword_label_CONSORT_11b,
    keyword_label_CONSORT_12a,
    keyword_label_CONSORT_12b,
    keyword_label_CONSORT_3a,
    keyword_label_CONSORT_3b,
    keyword_label_CONSORT_4a,
    keyword_label_CONSORT_4b,
    keyword_label_CONSORT_5,
    keyword_label_CONSORT_6a,
    keyword_label_CONSORT_6b,
    keyword_label_CONSORT_7a,
    keyword_label_CONSORT_7b,
    keyword_label_CONSORT_8a,
    keyword_label_CONSORT_8b,
    keyword_label_CONSORT_9,
    regex_10,
    regex_11a,
    regex_11b,
    regex_12a,
    regex_12b,
    regex_3a,
    regex_3b,
    regex_4a,
    regex_4b,
    regex_5,
    regex_6a,
    regex_6b,
    regex_7a,
    regex_7b,
    regex_8a,
    regex_8b,
    regex_9]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=snorkel_data_df)

  0%|          | 0/2460 [00:00<?, ?it/s]

KeyError: 'highest_frequency_label_in_top10'

In [5]:
from snorkel.labeling import LFAnalysis

L_train_list= L_train.tolist()
snorkel_data_df["Snorkel_LF"] = pd.Series(L_train_list)
LFAnalysis(L=L_train, lfs=lfs).lf_summary()


,j,Polarity,Coverage,Overlaps,Conflicts
similarity_lookup_CONSORT_10,0,[0],0.009719,0.008539,0.008470
similarity_lookup_CONSORT_11a,1,[1],0.000103,0.000093,0.000085
similarity_lookup_CONSORT_11b,2,[2],0.000015,0.000013,0.000013
similarity_lookup_CONSORT_12a,3,[3],0.024860,0.023942,0.011990
similarity_lookup_CONSORT_12b,4,[4],0.000015,0.000015,0.000015
similarity_lookup_CONSORT_3a,5,[5],0.000122,0.000100,0.000075
similarity_lookup_CONSORT_3b,6,[6],0.000040,0.000032,0.000032
similarity_lookup_CONSORT_4a,7,[7],0.003013,0.002777,0.001532
similarity_lookup_CONSORT_4b,8,[],0.000000,0.000000,0.000000
similarity_lookup_CONSORT_5,9,[9],0.021548,0.017942,0.012447


In [6]:
from snorkel.labeling.model import MajorityLabelVoter
from snorkel.utils import probs_to_preds
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=17, verbose=True)
label_model.fit(L_train=L_train, n_epochs=50, log_freq=100, seed=123)

In [7]:
probs_train = label_model.predict_proba(L=L_train)
print (probs_train.shape)

probs_train_list= probs_train.tolist()
snorkel_data_df["label_model_probability"] = pd.Series(probs_train_list)

prediction_list = label_model.predict(L=L_train)
prediction_labels = []
for item in prediction_list:
    if item == -1:
        prediction_labels.append("0")
    elif item == 0:
        prediction_labels.append("10")
    elif item == 1:
        prediction_labels.append("11a")
    elif item == 2:
        prediction_labels.append("11b")
    elif item == 3:
        prediction_labels.append("12a")
    elif item == 4:
        prediction_labels.append("12b")
    elif item == 5:
        prediction_labels.append("3a")
    elif item == 6:
        prediction_labels.append("3b")
    elif item == 7:
        prediction_labels.append("4a")
    elif item == 8:
        prediction_labels.append("4b")
    elif item == 9:
        prediction_labels.append("5")
    elif item == 10:
        prediction_labels.append("6a")
    elif item == 11:
        prediction_labels.append("6b")
    elif item == 12:
        prediction_labels.append("7a")
    elif item == 13:
        prediction_labels.append("7b")
    elif item == 14:
        prediction_labels.append("8a")
    elif item == 15:
        prediction_labels.append("8b")
    elif item == 16:
        prediction_labels.append("9")
        
snorkel_data_df["snorkel_prediction"] = prediction_labels
        

display(snorkel_data_df.head(5))

(599166, 17)


,index,PMID,sid,text,section_hierarchy,highest_frequency_label_in_top10,highest_similarity_label,Snorkel_LF,label_model_probability,snorkel_prediction
0,0,33062915,7,this study was performed as a randomized clini...,"['materials and methods', 'abstract']",0,3a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.018635784426180427, 0.017952697149294473, 0...",5
1,1,33062915,8,the primary outcome was clinical pregnancy rat...,"['materials and methods', 'abstract']",12a,6a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.02416296076560025, 0.02386135746099086, 0.0...",6a
2,2,33062915,42,this study was a randomized clinical trial per...,['2. materials and methods'],0,4b,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0
3,3,33062915,43,"the inclusion criteria were age <40 yr, at lea...",['2. materials and methods'],4a,4a,"[-1, -1, -1, -1, -1, -1, -1, 7, -1, -1, -1, -1...","[0.0004366703241002793, 0.0003711735626899982,...",4a
4,4,33062915,44,128 eligible women who were assigned for fet p...,['2. materials and methods'],0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.017961576374367287, 0.017171048529053268, 0...",4a


In [8]:
labels = ['10','11a','11b','12a','12b','3a','3b','4a','4b','5','6a','6b','7a','7b','8a','8b','9']

#eliminate rows that don't have predictions:
snorkel_data_df = snorkel_data_df[~snorkel_data_df["label_model_probability"].isnull()]
print (len(snorkel_data_df))

#get prediction by threshold 0.1
for index,row in snorkel_data_df.iterrows():
    predictions = []
    probs = row["label_model_probability"]
    for item in probs:
        if item>=0.45:
            prob_index = probs.index(item)
            predictions.append(labels[prob_index])
    if len(predictions) == 0:
        predictions.append('0')
        snorkel_data_df.at[index,"prediction_by_threshold_0.45"] = predictions
    else:
        snorkel_data_df.at[index,"prediction_by_threshold_0.45"] = predictions
    
display(snorkel_data_df.head(100))

599166


,index,PMID,sid,text,section_hierarchy,highest_frequency_label_in_top10,highest_similarity_label,Snorkel_LF,label_model_probability,snorkel_prediction,prediction_by_threshold_0.45
0,0,33062915,7,this study was performed as a randomized clini...,"['materials and methods', 'abstract']",0,3a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.018635784426180427, 0.017952697149294473, 0...",5,5
1,1,33062915,8,the primary outcome was clinical pregnancy rat...,"['materials and methods', 'abstract']",12a,6a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.02416296076560025, 0.02386135746099086, 0.0...",6a,[6a]
2,2,33062915,42,this study was a randomized clinical trial per...,['2. materials and methods'],0,4b,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0]
3,3,33062915,43,"the inclusion criteria were age <40 yr, at lea...",['2. materials and methods'],4a,4a,"[-1, -1, -1, -1, -1, -1, -1, 7, -1, -1, -1, -1...","[0.0004366703241002793, 0.0003711735626899982,...",4a,[4a]
4,4,33062915,44,128 eligible women who were assigned for fet p...,['2. materials and methods'],0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.017961576374367287, 0.017171048529053268, 0...",4a,[4a]
...,...,...,...,...,...,...,...,...,...,...,...
95,95,27590668,69,if a subsequent call was missed the crpc would...,"['experimental group', 'procedures', 'methods']",0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0]
96,96,27590668,70,the topics of each conversation were not prede...,"['experimental group', 'procedures', 'methods']",0,3b,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0]
97,97,27590668,71,call content was open ended and tailored to ea...,"['experimental group', 'procedures', 'methods']",0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0]
98,98,27590668,72,"however, mentors were trained (see below) in a...","['experimental group', 'procedures', 'methods']",0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0]


In [10]:
labels = ['10','11a','11b','12a','12b','3a','3b','4a','4b','5','6a','6b','7a','7b','8a','8b','9']

#eliminate rows that don't have predictions:
snorkel_data_df = snorkel_data_df[~snorkel_data_df["label_model_probability"].isnull()]
print (len(snorkel_data_df))

#get prediction by threshold 0.1
for index,row in snorkel_data_df.iterrows():
    predictions = []
    probs = row["label_model_probability"]
    for item in probs:
        if item>=0.8:
            prob_index = probs.index(item)
            predictions.append(labels[prob_index])
    if len(predictions) == 0:
        predictions.append('0')
        snorkel_data_df.at[index,"prediction_by_threshold_0.8"] = predictions
    else:
        snorkel_data_df.at[index,"prediction_by_threshold_0.8"] = predictions
    
display(snorkel_data_df.head(100))

599166


,index,PMID,sid,text,section_hierarchy,highest_frequency_label_in_top10,highest_similarity_label,Snorkel_LF,label_model_probability,snorkel_prediction,prediction_by_threshold_0.45,prediction_by_threshold_0.8
0,0,33062915,7,this study was performed as a randomized clini...,"['materials and methods', 'abstract']",0,3a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.018635784426180427, 0.017952697149294473, 0...",5,5,0
1,1,33062915,8,the primary outcome was clinical pregnancy rat...,"['materials and methods', 'abstract']",12a,6a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.02416296076560025, 0.02386135746099086, 0.0...",6a,[6a],[0]
2,2,33062915,42,this study was a randomized clinical trial per...,['2. materials and methods'],0,4b,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0],[0]
3,3,33062915,43,"the inclusion criteria were age <40 yr, at lea...",['2. materials and methods'],4a,4a,"[-1, -1, -1, -1, -1, -1, -1, 7, -1, -1, -1, -1...","[0.0004366703241002793, 0.0003711735626899982,...",4a,[4a],[4a]
4,4,33062915,44,128 eligible women who were assigned for fet p...,['2. materials and methods'],0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.017961576374367287, 0.017171048529053268, 0...",4a,[4a],[0]
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,27590668,69,if a subsequent call was missed the crpc would...,"['experimental group', 'procedures', 'methods']",0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0],[0]
96,96,27590668,70,the topics of each conversation were not prede...,"['experimental group', 'procedures', 'methods']",0,3b,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0],[0]
97,97,27590668,71,call content was open ended and tailored to ea...,"['experimental group', 'procedures', 'methods']",0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0],[0]
98,98,27590668,72,"however, mentors were trained (see below) in a...","['experimental group', 'procedures', 'methods']",0,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.058823529411764705, 0.058823529411764705, 0...",0,[0],[0]


In [11]:
display(snorkel_data_df.head(2))
snorkel_data_df.to_csv("/efs/CONSORT2/data/snorkel_data/all_data_SNORKEL_RESULTS_keywords_sectionheaders_sentencesimilarity_threshold08_06072021.csv", index = False)

,index,PMID,sid,text,section_hierarchy,highest_frequency_label_in_top10,highest_similarity_label,Snorkel_LF,label_model_probability,snorkel_prediction,prediction_by_threshold_0.45,prediction_by_threshold_0.8
0,0,33062915,7,this study was performed as a randomized clini...,"['materials and methods', 'abstract']",0,3a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.018635784426180427, 0.017952697149294473, 0...",5,5,0
1,1,33062915,8,the primary outcome was clinical pregnancy rat...,"['materials and methods', 'abstract']",12a,6a,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.02416296076560025, 0.02386135746099086, 0.0...",6a,[6a],[0]
